In [ ]:
!pip install langchain-core
!pip install -U langchain-openai
!pip install langgraph

In [ ]:
import getpass
import os

# Get API key from user input and store it in the environment variable.
os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass()

# Setup environment variable.
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://cccworkshop.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"

# Import AzureChatOpenAI
from langchain_openai import AzureChatOpenAI

# Instantiate the AzureChatOpenAI model, retrieving values from environment variables.
model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Add threat ID
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
query = "Imagine you are Professor McGonagall at Hogwarts School of Witchcraft and Wizardry, giving a detailed, magical explanation of Large Language Models (LLMs) to a class of wizards and witches. Go into as much detail as possible, explaining how an LLM learns, how it generates responses, and why it can sometimes be mistaken. Describe it as if it’s a rare and ancient magical artifact or spell, weaving in magical metaphors, Hogwarts lore, and spellbinding explanations that make complex topics enchanting and easy to understand."

input_messages = [HumanMessage(query)]


output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


In [ ]:
query = "Imagine you are Professor McGonagall at Hogwarts School of Witchcraft and Wizardry, giving a detailed, magical explanation of Large Language Models (LLMs) to a class of wizards and witches. Go into as much detail as possible, explaining how an LLM learns, how it generates responses, and why it can sometimes be mistaken. Describe it as if it’s a rare and ancient magical artifact or spell, weaving in magical metaphors, Hogwarts lore, and spellbinding explanations that make complex topics enchanting and easy to understand."

input_messages = [HumanMessage(query)]


for chunk, metadata in app.stream(
    {"messages": input_messages},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="")

